In [19]:
%matplotlib widget

In [20]:
from graph import graph 
from state import state
import matplotlib.pyplot as plt
import numpy as np
import random
import pylab as pl
from IPython import display
import networkx as nx
from copy import deepcopy
import ipywidgets as widgets

In [21]:
#WARNING: remember if size is greater than possible permutations, then this function will run indefinately 
def population_maker(graph:graph,start_idx,size):
    choice = [i for i in range(graph.V) if i!=start_idx]
    population=set()
    while(len(population)<size):
        path = [start_idx] + random.sample(choice,graph.V-1) + [start_idx]
        population.add(state(graph,path))
    return population 

In [22]:
def pair_maker(population):
    pairs=set()
    n=len(population)
    #ranked pairing half and remaining are random
    for i in range(0,n,2):
        pairs.add((population[i],population[i+1]))
    while(len(pairs)<n):
        pairs.add(tuple(random.sample(population,2)))
    return pairs

In [23]:
def breeding(parent1,parent2):
    stidx = parent1.start
    gene1 = int(random.random() * len(parent1.path))
    gene2 = int(random.random() * len(parent2.path))

    startGene = min(gene1, gene2)
    endGene = max(gene1, gene2)

    cp1 = [parent1.path[i] for i in range(startGene,endGene) if parent1.path[i]!=stidx]
    cp2 = [item for item in parent2.path if item not in cp1 and item != stidx]

    return state(parent1.network, [stidx] + cp1 + cp2 + [stidx])

In [24]:
#We breed randomly, carry the elites across without any breeding. should correct this eventually
def breed_selected(pairs):
    children=set()
    for p1,p2 in pairs:
        offspring = breeding(p1,p2)
        while offspring in children:
            offspring = breeding(p1,p2)
        children.add(offspring)
    return sorted(children,key = lambda x : x.path_dist)
    # return children

In [25]:
def mutate(child,threshold):
    n = len(child.path)
    for startidx in range(1,n-1):
        if(random.random()<threshold):
            endidx = int(random.random()*(n-2))+1
            child.swap_city(startidx,endidx)
            break #this is crucial if you want controlled mutation.
    return child

In [26]:
def mutate_next_generation(population,children,threshold,elite_fraction):
    parent_cutoff = int(len(population)*elite_fraction)
    mutated = set(population[:parent_cutoff])
    n,count=len(children),0
    # while(count<n-parent_cutoff):
    while(len(mutated)<n):
        child = children[count]
        mut_child = mutate(child,threshold)
        if(mut_child not in mutated):
            mutated.add(mut_child)
            count+=1
    return sorted(mutated,key = lambda x : x.path_dist)

In [27]:
def print_pop(population,gen):
    print("current gen: ",gen)
    # for child in population:
    #     print(child)
    print(population[0])
    print("------------------------------------")

In [28]:
def show_graph(G,hero,threshold,gen,pos=None):
    Gnx = nx.Graph()
    curr_city = hero.path[0]
    for next_city in hero.path[1:]:
        #keep 999 instead of infinity to keep graph intact
        dist = 10**8 if next_city not in G.adj_list[curr_city] else G.adj_list[curr_city][next_city]
        s,e,w = curr_city , next_city , dist
        Gnx.add_edge(s,e,weight=w)
        curr_city=next_city
    if not pos:
        pos=nx.kamada_kawai_layout(Gnx) #change layout if needed
    labels = nx.get_edge_attributes(Gnx,'weight')
    # nx.draw(Gnx)
    colors=['lime']*G.V
    act2nx = dict(zip(Gnx.nodes(),range(G.V)))
    colors[act2nx[curr_city]] = 'red'

    # print(act2nx)
    # pdf = PdfPages('out.pdf')
    pl.figure(1,figsize=(16,12)) 
    nx.draw_networkx_edge_labels(Gnx,pos,edge_labels=labels)
    nx.draw_networkx(Gnx,pos,arrows=True,node_color=colors,node_size=100,width=0.5)
    pl.text(0.8, 0.9, 'Gen: {0} | Cost: {1} | M.Rate: {2}'.format(gen,hero.path_dist,threshold), fontsize = 18)
    pl.plot()
    display.clear_output(wait=True)
    display.display(pl.gcf())
    pl.clf()
    return pos
    # pdf.savefig(plt.gcf())
    # for node in visited:
    #     colors[act2nx[node]]='blue'
    #     nx.draw_networkx(Gnx,pos,node_color=colors,node_size=50,width=0.1)
    #     pdf.savefig(plt.gcf())
    # pdf.close()
    # plt.show()

# TSP with Genetic Algorithm

In [29]:
def run_tsp(obj):
    # G = graph('dantzig42-699.txt')
    # G = graph('att48_33523.txt')
    G = graph(filename_wid.value)

#     start_city_idx = 0
#     threshold = 0.05
#     elite_fraction = 0.3
#     gen_size = 20
#     generations = 100 # hardcoded rn
#     print_interval = generations//50

    start_city_idx = 0
    threshold = threshold_wid.value
    elite_fraction = elite_fraction_wid.value
    gen_size = gen_size_wid.value
    generations = generations_wid.value # hardcoded rn
    print_interval = generations//50

#     def obs_mut_rate(change):
#         global threshold
#         threshold = threshold_wid.value
#     threshold_wid.observe(obs_mut_rate)
    
    population = sorted(population_maker(G,start_city_idx,gen_size),key=lambda x:x.path_dist)

    hero = population[0]
    herodb=[]
    pos = None
    
    out_wid.clear_output()
    with out_wid:
        for gen in range(generations):
            #uncomment this for real time updating:----------------not working :(
            
#             threshold = threshold_wid.value
            
            
            hero = deepcopy(min(hero,population[0])) #remembers the best possible offspring we have ever bred. this will be returned
            #uncomment to print all children in a generation
            # print_pop(population,gen)
            pairs = pair_maker(population)
            children = breed_selected(pairs)
            hero = deepcopy(min(hero,children[0]))
            herodb.append(hero.path_dist)
            population = mutate_next_generation(population,children,threshold,elite_fraction)
            if(gen%print_interval==0):
                pos = show_graph(G,hero,threshold,gen,pos)



        print("best path found: ",hero)
        pl.figure(2,figsize=(10,8))
        pl.xlabel('Generations')
        pl.ylabel('Path Cost')
        pl.plot(range(generations),herodb,"b",label="hero stat")
        pl.show()

In [12]:
# run_tsp(None)

In [30]:
# start_city_idx = 0
# threshold = 0.05
# elite_fraction = 0.3
# gen_size = 20
# generations = 200 # hardcoded rn
# print_interval = generations//50

out_wid = widgets.Output(layout={'border': '1px solid black'})

generations_wid = widgets.IntText(value=2000,description="generations:")
gen_size_wid = widgets.IntText(value=100,description="gen size:")
threshold_wid = widgets.FloatSlider(min=0,max=1,step=0.01,description='mutation:',value=0.20)
elite_fraction_wid = widgets.FloatSlider(min=0,max=1,step=0.01,description='super elites:',value=0.40)
run_tsp_btn = widgets.Button(description='Start TSP')
filename_wid = widgets.Textarea(value='graph50.txt',placeholder='graph50.txt',description='InputFile:',disabled=False)
run_tsp_btn.on_click(run_tsp)
# label = widgets.Label('TSP with Genetic Algorithm')
t_box = widgets.VBox([filename_wid,run_tsp_btn])
l_box = widgets.VBox([generations_wid,gen_size_wid])
r_box = widgets.VBox([threshold_wid , elite_fraction_wid])
display.display(widgets.HBox([t_box,l_box,r_box]))
display.display(out_wid)

Output(layout=Layout(border='1px solid black'))

In [14]:
# display.display("TSP Problem",generations_wid,gen_size_wid,threshold_wid,elite_fraction_wid,run_tsp_btn)